In [81]:
import cv2
from deepface import DeepFace
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import gradio as gr
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO

In [82]:
# Spotify API setup
SPOTIPY_CLIENT_ID = '-'
SPOTIPY_CLIENT_SECRET = '-'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope='user-library-read user-top-read playlist-modify-private user-read-recently-played'))

In [83]:
# More complex feature mappings
emotion_feature_map = {
    'happy': {'valence': 0.8, 'energy': 0.8, 'danceability': 0.7, 'tempo': 120},
    'sad': {'valence': 0.2, 'energy': 0.3, 'danceability': 0.3, 'tempo': 70},
    'angry': {'valence': 0.3, 'energy': 0.9, 'danceability': 0.5, 'tempo': 140},
    'neutral': {'valence': 0.5, 'energy': 0.5, 'danceability': 0.5, 'tempo': 100},
    'surprise': {'valence': 0.6, 'energy': 0.7, 'danceability': 0.6, 'tempo': 110},
    'fear': {'valence': 0.2, 'energy': 0.4, 'danceability': 0.3, 'tempo': 90},
    'disgust': {'valence': 0.3, 'energy': 0.6, 'danceability': 0.4, 'tempo': 95}
}

age_feature_map = {
    'young': {'popularity': 80, 'acousticness': 0.2, 'instrumentalness': 0.1, 'loudness': -5},
    'adult': {'popularity': 60, 'acousticness': 0.4, 'instrumentalness': 0.3, 'loudness': -8},
    'senior': {'popularity': 40, 'acousticness': 0.6, 'instrumentalness': 0.5, 'loudness': -10}
}

gender_feature_map = {
    'Man': {'speechiness': 0.5, 'liveness': 0.4, 'mode': 1},
    'Woman': {'speechiness': 0.4, 'liveness': 0.5, 'mode': 0}
}

def get_age_category(age):
    if age < 30:
        return 'young'
    elif age < 60:
        return 'adult'
    else:
        return 'senior'

def get_audio_features(track_ids):
    all_features = []
    batch_size = 50  # Spotify allows up to 100 tracks per request, but we'll use 50 to be safe

    for i in range(0, len(track_ids), batch_size):
        batch = track_ids[i:i+batch_size]
        try:
            features = sp.audio_features(tracks=batch)
            all_features.extend([f for f in features if f is not None])
        except Exception as e:
            print(f"Error fetching audio features for batch {i//batch_size + 1}: {str(e)}")

    return all_features

def create_user_profile(emotion, age, gender):
    age_category = get_age_category(age)
    profile = {}
    profile.update(emotion_feature_map.get(emotion, {}))
    profile.update(age_feature_map.get(age_category, {}))
    profile.update(gender_feature_map.get(gender, {}))
    return profile

def get_user_listening_history():
    recent_tracks = sp.current_user_recently_played(limit=50)
    top_tracks_short = sp.current_user_top_tracks(limit=50, time_range='short_term')
    top_tracks_medium = sp.current_user_top_tracks(limit=50, time_range='medium_term')
    top_tracks_long = sp.current_user_top_tracks(limit=50, time_range='long_term')
    
    all_tracks = (
        [item['track']['id'] for item in recent_tracks['items']] +
        [track['id'] for track in top_tracks_short['items']] +
        [track['id'] for track in top_tracks_medium['items']] +
        [track['id'] for track in top_tracks_long['items']]
    )
    return list(set(all_tracks))  # Remove duplicates

def cluster_tracks(features, n_clusters=5):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(features)
    return clusters

def reduce_dimensions(features, n_components=3):
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    return reduced_features

def get_recommendations(emotion, age, gender):
    print("Creating user profile...")
    user_profile = create_user_profile(emotion, age, gender)
    print("User profile created:", user_profile)

    print("Getting user listening history...")
    user_tracks = get_user_listening_history()
    print("User tracks retrieved:", user_tracks)

    # Get audio features for user's tracks
    print("Getting audio features for user tracks...")
    tracks_features = get_audio_features(user_tracks)
    print("Audio features retrieved.")

    # Create a feature matrix
    feature_names = ['valence', 'energy', 'danceability', 'tempo', 'popularity', 'acousticness', 'instrumentalness', 'loudness', 'speechiness', 'liveness', 'mode']
    feature_matrix = np.array([[track.get(feature, 0) for feature in feature_names] for track in tracks_features])
    print("Feature matrix created:", feature_matrix)

    # Normalize the feature matrix
    scaler = MinMaxScaler()
    normalized_feature_matrix = scaler.fit_transform(feature_matrix)
    print("Feature matrix normalized.")

    # Reduce dimensions for visualization (if needed)
    reduced_features = reduce_dimensions(normalized_feature_matrix)
    print("Dimensions reduced.")

    # Create user profile vector
    user_vector = np.array([[user_profile.get(feature, 0.5) for feature in feature_names]])
    normalized_user_vector = scaler.transform(user_vector)
    print("User profile vector created and normalized:", normalized_user_vector)

    # Calculate cosine similarity
    similarities = cosine_similarity(normalized_user_vector, normalized_feature_matrix)[0]
    print("Cosine similarity calculated:", similarities)

    # Get top 10 most similar tracks
    top_similar_indices = similarities.argsort()[-10:][::-1]
    top_similar_tracks = [user_tracks[i] for i in top_similar_indices]

    print("Top similar tracks retrieved:", top_similar_tracks)

    # Get recommendations based on seed tracks and user profile
    seed_tracks = top_similar_tracks[:5]  # Use top similar tracks as seed
    recommendations = sp.recommendations(seed_tracks=seed_tracks, limit=20,
                                         target_valence=user_profile['valence'],
                                         target_energy=user_profile['energy'],
                                         target_danceability=user_profile['danceability'],
                                         target_tempo=user_profile['tempo'])

    print("Recommendations retrieved.")
    return [track['id'] for track in recommendations['tracks']]


def create_playlist(track_ids):
    user_id = sp.me()['id']
    playlist = sp.user_playlist_create(user_id, 'Advanced Emotion-Age-Gender Based Recommendations', public=False)
    sp.user_playlist_add_tracks(user_id, playlist['id'], track_ids)
    return playlist['external_urls']['spotify']

def capture_image():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    return frame

def analyze_facial_features(frame):
    print("Analyzing facial features...")
    analysis = DeepFace.analyze(frame, actions=['emotion', 'age', 'gender'], enforce_detection=False)
    print("Facial analysis result:", analysis)
    
    # Check if analysis is a list and take the first item if so
    if isinstance(analysis, list):
        analysis = analysis[0]
    
    emotion = analysis['dominant_emotion']
    age = analysis['age']
    gender = analysis['dominant_gender']
    
    # Additional facial feature analysis
    face_features = analysis['region']
    face_width = face_features['w']
    face_height = face_features['h']
    face_ratio = face_width / face_height
    
    return emotion, age, gender, face_ratio

def analyze_and_recommend():
    frame = capture_image()
    try:
        print("Analyzing facial features...")
        emotion, age, gender, face_ratio = analyze_facial_features(frame)
        print(f"Facial features analyzed: Emotion: {emotion}, Age: {age}, Gender: {gender}, Face Ratio: {face_ratio:.2f}")
        
        # Adjust recommendations based on face ratio (just an example)
        if face_ratio > 1.1:
            # Wider face, might prefer more energetic music
            emotion_feature_map[emotion]['energy'] = min(1.0, emotion_feature_map[emotion].get('energy', 0.5) + 0.1)
        elif face_ratio < 0.9:
            # Longer face, might prefer calmer music
            emotion_feature_map[emotion]['energy'] = max(0.0, emotion_feature_map[emotion].get('energy', 0.5) - 0.1)
        
        print("Getting recommendations...")
        track_ids = get_recommendations(emotion, age, gender)
        if not track_ids:
            print("No recommendations found.")
            return

        print("Creating playlist...")
        playlist_url = create_playlist(track_ids)
        
        print("Getting additional track information...")
        tracks = sp.tracks(track_ids)['tracks']
        genres = set()
        for track in tracks:
            artist_id = track['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres.update(artist_info.get('genres', []))
        
        top_genres = ', '.join(list(genres)[:5])  # Top 5 genres
        
        print(f"Emotion: {emotion}, Age: {age}, Gender: {gender}")
        print(f"Face Ratio: {face_ratio:.2f}")
        print(f"Top Genres: {top_genres}")
        print(f"Playlist: {playlist_url}")

    except Exception as e:
        print(f"Error during recommendation process: {str(e)}")
        import traceback
        traceback.print_exc()

In [87]:
def capture_and_analyze_image():
    frame = capture_image()
    try:
        emotion, age, gender, face_ratio = analyze_facial_features(frame)
        
        # Adjust recommendations based on face ratio (just an example)
        if face_ratio > 1.1:
            emotion_feature_map[emotion]['energy'] = min(1.0, emotion_feature_map[emotion].get('energy', 0.5) + 0.1)
        elif face_ratio < 0.9:
            emotion_feature_map[emotion]['energy'] = max(0.0, emotion_feature_map[emotion].get('energy', 0.5) - 0.1)
        
        # Get recommendations
        track_ids = get_recommendations(emotion, age, gender)
        if not track_ids:
            return frame, "No recommendations found."
        
        # Create playlist
        playlist_url = create_playlist(track_ids)
        
        # Get additional track information
        tracks = sp.tracks(track_ids)['tracks']
        genres = set()
        for track in tracks:
            artist_id = track['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres.update(artist_info.get('genres', []))
        
        top_genres = ', '.join(list(genres)[:5])  # Top 5 genres
        
        # Format output
        result_text = f"Emotion: {emotion}\nAge: {age}\nGender: {gender}\nTop Genres: {top_genres}\nPlaylist: {playlist_url}"
        
        return frame, result_text

    except Exception as e:
        return frame, f"Error during recommendation process: {str(e)}"


In [88]:
# Define Gradio interface
iface = gr.Interface(
    fn=capture_and_analyze_image,
    inputs=None,
    outputs=["image", "text"],
    title="Facial Emotion-Based Music Recommendations",
    description="Capture your image to get personalized music recommendations based on your facial expression."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Analyzing facial features...


Action: gender: 100%|██████████| 3/3 [00:00<00:00,  6.46it/s] 


Facial analysis result: [{'emotion': {'angry': 97.0670223236084, 'disgust': 5.459363023874175e-05, 'fear': 0.051638646982610226, 'happy': 2.2770216688513756, 'sad': 0.6001338362693787, 'surprise': 1.3682682720173034e-05, 'neutral': 0.004115197953069583}, 'dominant_emotion': 'angry', 'region': {'x': 0, 'y': 0, 'w': 640, 'h': 480, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'age': 33, 'gender': {'Woman': 5.925409868359566, 'Man': 94.07459497451782}, 'dominant_gender': 'Man'}]
Creating user profile...
User profile created: {'valence': 0.3, 'energy': 1.0, 'danceability': 0.5, 'tempo': 140, 'popularity': 60, 'acousticness': 0.4, 'instrumentalness': 0.3, 'loudness': -8, 'speechiness': 0.5, 'liveness': 0.4, 'mode': 1}
Getting user listening history...
User tracks retrieved: ['5PEZxy0cvFiWc9A2CcWipW', '7lQ8MOhq6IN2w8EYcFNSUk', '5QJMiuwSROC3Kdirx7VBgX', '6GpyfvNqCPGdwWNgmSLp3D', '0WbMK4wrZ1wFSty9F7FCgu', '4JHP2bJv5CZbUJ8D7NcmmG', '5OFlp1fOrfiSXOjw4wuSgy', '2vZxL6vruuBFLlUAVeGTQf